# D209 Performance Assessment - Task 1: Classification Analysis & Task 2: Predictive Analysis
### by Bader Ale 

# ▶ Research Question
Our research questions is as follows: __Given certain patient characteristics, can we classify whether a patient is hypertensive or not__.

In [ ]:
# Importing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.interactiveshell import InteractiveShell # Importing so we can run multiple lines in one cell
InteractiveShell.ast_node_interactivity = "all" # Code so multiple lines in one cell can be ran simultaenously 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing original dataset
df = pd.read_csv('F:/GitHub Repos/WGU_MSDA/D209_Data Mining I/medical_clean.csv')

# ▶ Data Cleaning  

In [ ]:
df.head()

In [ ]:
df.drop(['CaseOrder','Customer_id','Interaction', 'UID', 'City', 'State', 
        'County', 'Zip', 'Lat', 'Lng', 'Population', 'TimeZone', 'Job', 
        'Children','ReAdmis', 'Full_meals_eaten','vitD_supp', 'Soft_drink',
        'Additional_charges', 'Item1', 'Item2', 'Item3', 'Item4', 'Item5',
        'Item6', 'Item7', 'Item8'], axis=1, inplace=True) 

In [ ]:
df.head()

## Detection and Treatment of Nulls

In [ ]:
# Checking for null values
df.isnull().sum()

## Detection and Treatment of Duplicated Values

In [ ]:
# Checking for duplicates
df.duplicated().value_counts()

## Detection and Treatment of Outliers

In [ ]:
df.dtypes

In [ ]:
# Scaling data to make it easier to visualize outliers
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df[['Age', 'Income', 'VitD_levels', 'Doc_visits', 'Initial_days', 'TotalCharge']])
df_scaled = pd.DataFrame(scaler.fit_transform(df[['Age', 'Income', 'VitD_levels', 'Doc_visits', 'Initial_days', 'TotalCharge']]),
                         columns=['Age', 'Income', 'VitD_levels', 'Doc_visits', 'Initial_days', 'TotalCharge'])

In [ ]:
# Visualizing Outliers
sns.boxplot(df_scaled)
plt.title('Numeric Variables')
plt.ylabel('Count(Normalized)')
plt.xlabel('Explanatory Variables');

In [ ]:
# Removing Outliers using z-scores
import scipy.stats as stats

#Creating New columns for z-scores
df['Income_z_Scores'] = stats.zscore(df['Income'])
df['VitD_level_z_Scores'] = stats.zscore(df['VitD_levels'])

# Removing records with z-scores -3 < z < 3
df = df[(df['Income_z_Scores'] > -3) & (df['Income_z_Scores'] < 3) & (df['VitD_level_z_Scores'] > -3) & (df['VitD_level_z_Scores'] < 3)]
df.drop(['Income_z_Scores', 'VitD_level_z_Scores'], axis=1, inplace=True)

In [ ]:
df.head()

# ▶ Data Wrangling  

In [ ]:
df.dtypes

In [ ]:
# Using .get_dummies() to re-express nominal variables
df = pd.get_dummies(data=df, columns=['Area','Marital','Gender', 'Initial_admin','HighBlood',
                                      'Stroke','Overweight', 'Arthritis','Diabetes', 'Hyperlipidemia', 
                                      'BackPain', 'Anxiety', 'Allergic_rhinitis','Reflux_esophagitis', 
                                      'Asthma','Services'], drop_first=True)

In [ ]:
# Using Ordinal Encoding for ordinal variables
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
complication_encoded = enc.fit_transform(df[['Complication_risk']])
df['Complication_risk'] = complication_encoded

In [ ]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
df.head()

In [ ]:
# Exporting cleaned dataset
cleaned_dataset = df.to_csv('cleaned_dataset.csv', index = False)

# ▶ Classification Analysis using k-Nearest Neighbor

In [ ]:
X = df.drop('HighBlood_Yes', axis=1)
y = df['HighBlood_Yes']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Splitting data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0, stratify=y)

In [ ]:
# Exporting train and test data to CSV
X_train.to_csv('X_Train.csv',index=False)
X_test.to_csv('X_Test.csv', index=False)
np.savetxt('y_train.csv', y_train, delimiter=",", header="HighBlood_Yes", fmt="%.2f", comments="")
np.savetxt('y_test.csv', y_test, delimiter=",", header="HighBlood_Yes", fmt="%.2f", comments="")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,y_train)

In [ ]:
knn.predict(X_test.values)

In [ ]:
print(f'The prediction score for the model is: {knn.score(X_test.values, y_test)}')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

print(f'Confusion Matrix: \n{confusion_matrix(y_test,knn.predict(X_test.values))}')
print(f'\nClassification Report for the Model: \n{classification_report(y_test, knn.predict(X_test.values))}')
print(f'\nThe area under the curve (AUC) is: {roc_auc_score(y_test, knn.predict(X_test.values))}')


# ▶ Predictive Analysis using Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

# Define a range of values for max_depth to test
max_depths = range(1, 10)

# Loop over max_depth values and evaluate model performance
best_accuracy = 0
best_max_depth = 0

for max_depth in max_depths:
    # Train a DecisionTreeClassifier model with current max_depth
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(X_train, y_train)
    
    # Make predictions on test set and calculate accuracy
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Update best accuracy and max_depth if current model is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_depth = max_depth

# Print best max_depth and corresponding accuracy
print("Best Max_Depth:", best_max_depth)
print("Accuracy:", best_accuracy)

In [ ]:
# Mean Squared Error
mean_squared_error(clf.predict(X_test), y_test)